In [1]:
# Install llama.cpp
! git clone https://github.com/ggerganov/llama.cpp
! cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
! pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 23479, done.
remote: Counting objects: 100% (9431/9431), done.
remote: Compressing objects: 100% (550/550), done.
remote: Total 23479 (delta 9208), reused 8884 (delta 8881), pack-reused 14048
Receiving objects: 100% (23479/23479), 34.95 MiB | 29.00 MiB/s, done.
Resolving deltas: 100% (16704/16704), done.
Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations

In [2]:
MODEL_ID = "MadMarx37/deepseek-coder-1.3b-python-peft"

# Download model
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
! apt-get install git-lfs
#the two commands above (bash and apt-get) might need sudo based on where you're running them
! git lfs install
! git clone https://huggingface.co/{MODEL_ID}

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.11
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 55 not upgraded.
Need to get 7420 kB of archives.
After this operation, 16.5 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.5.1 [7420 kB]
Fetched 7420 kB in 0s (16.6 MB/s)
debconf: delaying package configuration, since apt-

In [9]:
MODEL_NAME = MODEL_ID.split('/')[-1]

# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"

#vocab-type option is required for deepseek-coder models (https://github.com/ggerganov/llama.cpp/issues/6690#issuecomment-2058098140)
! python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16} --vocab-type bpe --pad-vocab

Loading model file deepseek-coder-1.3b-python-peft/model.safetensors
params = Params(n_vocab=32256, n_embd=2048, n_layer=24, n_ctx=16384, n_ff=5504, n_head=16, n_head_kv=16, n_experts=None, n_experts_used=None, f_norm_eps=1e-06, rope_scaling_type=<RopeScalingType.LINEAR: 'linear'>, f_rope_freq_base=100000, f_rope_scale=4.0, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('deepseek-coder-1.3b-python-peft'))
Loaded vocab file PosixPath('deepseek-coder-1.3b-python-peft/tokenizer.json'), type 'bpe'
Vocab info: <BpeVocab with 32000 base tokens and 22 added tokens>
Special vocab info: <SpecialVocab with 31757 merges, special tokens {'bos': 32013, 'eos': 32014, 'pad': 32014}, add special tokens {'bos': True, 'eos': False}>
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting 

In [10]:
QUANTIZATION_METHODS = ["q5_k_m"]

for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/quantize {fp16} {qtype} {method}

main: build = 2770 (952d03db)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'deepseek-coder-1.3b-python-peft/deepseek-coder-1.3b-python-peft.fp16.bin' to 'deepseek-coder-1.3b-python-peft/deepseek-coder-1.3b-python-peft.Q5_K_M.gguf' as Q5_K_M
llama_model_loader: loaded meta data with 25 key-value pairs and 219 tensors from deepseek-coder-1.3b-python-peft/deepseek-coder-1.3b-python-peft.fp16.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32256
llama_model_loader: - kv   3:                       llama.context_length u32              = 16384
llama_model_loader: - kv   4:           

In [14]:
import os

model_list = [file for file in os.listdir(MODEL_NAME) if "gguf" in file]

prompt = input("Enter your prompt: ")
chosen_method = input("Name of the model (options: " + ", ".join(model_list) + "): ")

# Verify the chosen method is in the list
if chosen_method not in model_list:
    print("Invalid name")
else:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/main -m {qtype} -n 128 --color -ngl 35 -p "{prompt}"

Enter your prompt:  # Split data into train and test sets from sklearn.model_selection import train_test_split X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Initialize and train a machine learning model (e.g., Random Forest)
Name of the model (options: deepseek-coder-1.3b-python-peft.Q5_K_M.gguf):  deepseek-coder-1.3b-python-peft.Q5_K_M.gguf


Log start
main: build = 2770 (952d03db)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1714466025
llama_model_loader: loaded meta data with 26 key-value pairs and 219 tensors from deepseek-coder-1.3b-python-peft/deepseek-coder-1.3b-python-peft.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32256
llama_model_loader: - kv   3:                       llama.context_length u32              = 16384
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   5:                          llama.block_count u32            

In [15]:
! pip install -q huggingface_hub
from huggingface_hub import create_repo, HfApi
import getpass

hf_token = getpass.getpass()

api = HfApi()
username = "MadMarx37"

# Create empty repo
create_repo(
    repo_id = f"{username}/{MODEL_NAME}-GGUF",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)

# Upload gguf files
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    allow_patterns=f"*.gguf",
    token=hf_token
)

 ········


deepseek-coder-1.3b-python-peft.Q5_K_M.gguf:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MadMarx37/deepseek-coder-1.3b-python-peft-GGUF/commit/870609b1eaa1cb6924bb9e2a61eb1ff4ef0c2aab', commit_message='Upload folder using huggingface_hub', commit_description='', oid='870609b1eaa1cb6924bb9e2a61eb1ff4ef0c2aab', pr_url=None, pr_revision=None, pr_num=None)